In [1]:
# Stacking Layers with .add()
from keras.layers import Dense
# Sequential model
from keras.models import Sequential
from keras.losses import categorical_crossentropy
from keras.optimizers import SGD
import tensorflow as tf
from keras import layers

2023-10-16 13:57:01.627652: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-16 13:57:01.780722: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-16 13:57:01.781465: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-16 13:57:05.918977: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
!curl -O https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  786M  100  786M    0     0  10.6M      0  0:01:13  0:01:13 --:--:-- 15.4M


In [3]:
!unzip -q kagglecatsanddogs_5340.zip
!ls

replace PetImages/Cat/0.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
!ls PetImages

In [ ]:
## Filter out corrupted images
import os

num_skipped = 0
for folder_name in ('Cat', 'Dog'):
    folder_path = os.path.join('PetImages', folder_name)
    for fname in os.listdir(folder_path):
        fpath = os.path.join(folder_path, fname)
        try:
            fobj = open(fpath, 'rb')
            is_jfif = tf.compat.as_bytes('JFIF') in fobj.peek(10)
        finally:
            fobj.close()
            
        if not is_jfif:
            num_skipped += 1
            # Now delete corrupted image
            os.remove(fpath)
            
print("Deleted %d images:"% num_skipped)

In [ ]:
### generate the Dataset

image_size = (200, 200)
batch_size = 128

train_dataset, val_dataset = tf.keras.utils.image_dataset_from_directory(
"PetImages", validation_split = 0.2, subset = 'both', seed = 12,
image_size = image_size, batch_size = batch_size)

In [ ]:
# Now Visualize the images
import matplotlib.pyplot as plt
plt.figure(figsize = (10, 5))
for img, label in train_dataset.take(1):
    for i in range(8):
        ax = plt.subplot(2,4,i+1)
        plt.imshow(img[i].numpy().astype('uint8'))
        plt.title(int(label[i]))
        plt.axis('off')


In [ ]:
data_augmentation = Sequential([layers.RandomFlip('horizontal'), 
            layers.RandomRotation(0.1)])
print(data_augmentation)

In [ ]:
plt.figure(figsize = (10, 5))
for img, _ in train_dataset.take(1):
    for i in range(8):
        augmented_img = data_augmentation(img)
        ax = plt.subplot(2,4, i+1)
        plt.imshow(augmented_img[0].numpy().astype('uint8'))
        plt.axis('off')

In [ ]:
model = Sequential()
model.add(Dense(units = 64, activation = 'relu'))
model.add(Dense(units = 10, activation = 'softmax'))

In [ ]:
#Once the model looks good than configure the learning process
# with .compile()
model.compile(loss = "categorical_crossentropy",
             optimizer = 'sgd',
             metrics = ['accuracy'])

In [ ]:
model.compile(optimizer = SGD(learning_rate = 0.01, momentum = 0.9, 
                             nesterov = True),
             loss = categorical_crossentropy)

In [ ]:
# to prepare an Optimizer
optimizer = tf.keras.optimizers.Adam()

# Prepare Loss function
loss = tf.keras.losses.kl_divergence

In [ ]:
# Iterate over the batches of the dataset
for ip,op in train_dataset:
    # Open a gradiantTape
    with tf.GradiantTape() as tape:
        # Forward Pass
        predictions = model(ip)
        # Compute the loss value for this batch
        loss_value = loss(op, predictions)
    # Get gradiants of loss with respect to the weights
    gardiants = tape.gradiant(loss_value, model.trainable_weights)
    
    # Now, update the weights of the model
    optimizer.apply_gradiants(zip(gradiants, model.trainabel_weights))